In [289]:
import pickle
import tensorflow as tf

# Memuat data dan model yang diperlukan
folder = './model/'
user_id = 'c74c14bf-d6ae-455d-842c-6c02cd638349'
k = 10
cb_df = pickle.load(open(folder + 'cb_df.pkl', 'rb'))
collab_df = pickle.load(open(folder + 'collab_df.pkl', 'rb'))
user_id_encoded = pickle.load(open(folder + 'user_id_encoded.pkl', 'rb'))
farmer_id_encoded = pickle.load(open(folder + 'farmer_id_encoded.pkl', 'rb'))
model = tf.keras.models.load_model(folder + '/model_collab')
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [290]:
cb_df[['farmer_id', 'nama', 'jenis_sawah']]

,farmer_id,nama,jenis_sawah
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,datarantinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,irigasi
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi
3,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,dataranrendah
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,ladang
5,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,ladang
6,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,dataranrendah


In [291]:
collab_df

,user_id,farmer_id,rating,user,farmer
0,2bd66dd6-5ac6-40ad-968e-1271da13db32,1000a253-a43c-4ab3-91f7-8c370e01b9c4,4,0,0
1,b0af16ed-0633-42a2-afc7-2c5e48db5a48,2345a62a-8d3a-4f7b-aad5-7e50670a817b,4,1,1
5,c74c14bf-d6ae-455d-842c-6c02cd638349,cecb6e32-8f30-4806-a16c-a6073966ad65,5,2,5
2,2bd66dd6-5ac6-40ad-968e-1271da13db32,58c5ee91-37b5-4818-8815-872eae86f21c,5,0,2
4,b0af16ed-0633-42a2-afc7-2c5e48db5a48,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,4,1,4
3,c74c14bf-d6ae-455d-842c-6c02cd638349,7488ab52-bc5b-4ff2-9427-a01b64145c6c,3,2,3
6,c74c14bf-d6ae-455d-842c-6c02cd638349,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,2,2,6


In [292]:
model.summary()

Model: "recommender_net_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_172 (Embedding)   multiple                  300       
                                                                 
 embedding_173 (Embedding)   multiple                  3         
                                                                 
 embedding_174 (Embedding)   multiple                  700       
                                                                 
 embedding_175 (Embedding)   multiple                  7         
                                                                 
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________


In [293]:
user_id_encoded

{'2bd66dd6-5ac6-40ad-968e-1271da13db32': 0,
 'b0af16ed-0633-42a2-afc7-2c5e48db5a48': 1,
 'c74c14bf-d6ae-455d-842c-6c02cd638349': 2}

In [294]:
farmer_id_encoded

{'1000a253-a43c-4ab3-91f7-8c370e01b9c4': 0,
 '2345a62a-8d3a-4f7b-aad5-7e50670a817b': 1,
 '58c5ee91-37b5-4818-8815-872eae86f21c': 2,
 '7488ab52-bc5b-4ff2-9427-a01b64145c6c': 3,
 'a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7': 4,
 'cecb6e32-8f30-4806-a16c-a6073966ad65': 5,
 'e9ef08e2-9d2d-49d4-8934-7f259cbbf867': 6}

In [295]:
import numpy as np
import tensorflow as tf


def get_collab_recommendations(user_id, items, collab_df, model, user_id_encoded, farmer_id_encoded, k=5):
  farmer_reviewed_by_user = collab_df[collab_df['user_id'] == user_id]
  farmer_not_reviewed = items[~items['farmer_id'].isin(
    farmer_reviewed_by_user['farmer_id'])]

  farmer_not_reviewed = list(set(farmer_not_reviewed['farmer_id'].tolist(
  )).intersection(set(farmer_id_encoded.keys())))

  farmer_not_reviewed = [[user_id_encoded.get(user_id), farmer_id_encoded.get(
    farmer_id)] for farmer_id in farmer_not_reviewed]

  user_farmer_array = np.array(farmer_not_reviewed)
  user_farmer_array = tf.convert_to_tensor(
    user_farmer_array, dtype=tf.int64)  # Convert to tf.int64

  ratings = model.predict(user_farmer_array).flatten()

  top_ratings_indices = ratings.argsort()[-k:][::-1]

  farmer_encoded = {i: x for i, x in enumerate(farmer_id_encoded)}
  recommended_farmer_ids = [farmer_encoded.get(
    farmer_not_reviewed[x][1]) for x in top_ratings_indices]

  recommended_farmer = items[items['farmer_id'].isin(recommended_farmer_ids)]

  return recommended_farmer

In [296]:
get_collab_recommendations(user_id=user_id, items=cb_df, collab_df=collab_df, model=model, user_id_encoded=user_id_encoded, farmer_id_encoded=farmer_id_encoded, k=10)

1/1 [==============================] - 0s 139ms/step


,farmer_id,nama,jenis_sawah
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,datarantinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,irigasi
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,ladang
